In [ ]:
import sys
!{sys.executable} -m pip install rdflib==7.4.0 pandas==2.3.3

In [68]:
from os import getcwd
from os.path import join, dirname
folder = dirname(getcwd())

## Initialise the Knowledge Graph

Download the HEAT-KG dump from Zenodo and place it in the `data` folder. Then, read the HEAT-KG from the file and load it in memory to execute SPARQL queries against it. (You can edit the notebook to execute the queries against a SPARQL endpoint.)

In [ ]:
import urllib.request
import os

# Download HEAT-KG file from Zenodo
# Check https://doi.org/10.5281/zenodo.17805802 and find the link to the latest release of the heat-kg.ttl file and, if different, change the URL below
url = "https://zenodo.org/records/17805803/files/heat-kg.ttl"  
output_path = join(folder, "data/heat-kg.ttl")

# Create data directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Download the file
print(f"Downloading from {url}...")
urllib.request.urlretrieve(url, output_path)
print(f"File downloaded successfully to {output_path}")

In [ ]:
from rdflib import Graph

# Load HEAT-KG data from the downloaded file
g = Graph()
rdf_file = join(folder, "data/heat-kg.ttl")
g.parse(rdf_file, format="ttl")

### Run SPARQL query against the HEAT-KG

The following function takes as input a SPARQL query from the "queries" folder and runs it against the HEAT-KG. To run the query, just invoke the execute_query function passing the name of the query file (usually a .rq file); if the query is to retrieve details about a specific URI, you can pass it as second parameter. If you want to run a different query, add the query file to the folder or modify the code according to your needs.

In [80]:
import pandas as pd
from IPython.display import display

# Imposta opzioni per visualizzazione completa
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


def execute_query(query_filename, uri=None):
    """
    Execute a SPARQL query from the queries folder and display results as an interactive table.
    
    Parameters:
    query_filename (str): Name of the query file in the queries folder (e.g., 'default.rq')
    uri (str, optional): URI to substitute for any SPARQL variable indicated as "?_something_iri" in the query
    """
    # Read query from file
    query_path = join(folder, "queries", query_filename)
    with open(query_path, 'r') as f:
        sparql_query = f.read()
    
    # If URI is provided, substitute all ?_*_iri variables with the URI
    if uri:
        import re
        # Find all variables matching pattern ?_*_iri and replace with the provided URI
        sparql_query = re.sub(r'\?\w*_iri\b', f'<{uri}>', sparql_query)
        print(f"Substituting URI: {uri}")
    
    print(f"Executing query: {query_filename}")
    print("=" * 50)
    
    # Execute the SPARQL query
    results = g.query(sparql_query)
    
    # Convert results to DataFrame for better display
    rows = []
    for row in results:
        rows.append({col: str(row[col]) for col in results.vars})
    
    df = pd.DataFrame(rows)
    
    return df

## HEAT-KG Exploration

The HEAT-KG contains information about Human Evaluation and Assessment tools, which are described with the respective ontology; if you want to learn more about the underlying model, check the HEAT ontology at https://w3id.org/heat.

Since there are different ways to search for relevant information within the HEAT-KG, hereafter we exemplify potential paths to explore it.

### Exploration starting from a Reference Theory

You may already be familiar with some user research theories and therefore you may already know you want to start from there. Say, for example, that you want to apply the Technology Acceptance Model (TAM) to evaluate whether your users are willing to adopt a specific digital system or application.

In this case you can start from running the query to retrieve all reference theories, to see what is available and to identify the URI for TAM.

In [81]:
# Execute the query to get all reference theories
execute_query('all-theories.rq')

Executing query: all-theories.rq


,theory,theoryLabel,theoryDescription
0,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/30322e9e-e88d-4e7c-9929-73b385b54f5d,Theory of Planned Behavior (TPB),"The Theory of Planned Behaviour (TPB) is a psychological model that explains how an individual’s attitude toward a behavior, subjective norms, and perceived behavioral control together shape their intention to perform the behavior, which in turn predicts actual behavior"
1,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/b1ec406a-31ff-4e46-8dac-821e6051adfe,Student Learning and Satisfaction in Online Learning Environments (SLS-OLE),The Student Learning and Satisfaction in Online Learning Environments (SLS-OLE) is a model designed to measure students' perceptions of their learning and satisfaction in asynchronous online courses
2,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/ec94234a-93f4-4ca4-b2e0-a64e93e33932,Anxiety Artificial Intelligence Scale (AIAS),The Artificial Intelligence Anxiety Scale (AIAS) is a model designed to assess the degree of anxiety or concern individuals experience regarding artificial intelligence
3,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/df9b0e8a-9819-420e-8f08-d6e478f789c1,Technology Acceptance Model (TAM),The Technology Acceptance Model (TAM) is a model designed to explains how users come to accept and use a technology or system
4,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/9ab54867-f2c0-4ee1-8bb0-8729e46eb906,Checklist Individual Strength (CIS),The Checklist Individual Strength (CIS) is a model designed to assess different aspects of fatigue
5,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/78427716-902a-49e0-b182-09c68f171dd5,Attitudes towards Artificial Intelligence Scale (ATTARI),The Attitudes Towards Artificial Intelligence Scale (ATTARI) is a model designed to measure people's attitudes toward artificial intelligence
6,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/f70447e2-33fa-41f5-b21f-9eede05baa04,Fatigue Assessment Scale (FAS),The Fatigue Assessment Scale (FAS) is a model designed to evaluate symptoms of chronic fatigue
7,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/2d002741-21fd-4f2c-bf4c-a549e54cbafa,System Usability Scale (SUS),"The System Usability Scale (SUS) is a model designed to measure the usability of a system, product, or service"
8,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/ca2d2ffb-416c-4b0e-94ee-2dcc4e052917,Computer Self Efficacy (CSE),"Computer Self-Efficacy (CSE) is a model designed to evaluate an individual's belief in their ability to effectively use computers or digital technologies, based on Bandura's Social Cognitive Theory"
9,https://knowledge.c-innovationhub.com/heat/ReferenceTheory/12ea78b9-865a-47d7-b8a3-0517880ef2db,Perceived Stress Scale (PSS),"The Perceived Stress Scale (PSS) is a model designed to measure psychological stress, as the degree to which situations in one's life are appraised as stressful"


There are indeed different interesting theories that could help us. Let's stick to TAM for now: from the above results we know that the TAM model has the following URI <https://knowledge.c-innovationhub.com/heat/ReferenceTheory/df9b0e8a-9819-420e-8f08-d6e478f789c1>. Let's query the HEAT-KG to see which tools implement the TAM reference theory.

In [83]:
# Execute the query to get all tools implementing the TAM theory
execute_query('tools-of-theory.rq','https://knowledge.c-innovationhub.com/heat/ReferenceTheory/df9b0e8a-9819-420e-8f08-d6e478f789c1')

Substituting URI: https://knowledge.c-innovationhub.com/heat/ReferenceTheory/df9b0e8a-9819-420e-8f08-d6e478f789c1
Executing query: tools-of-theory.rq


,theoryTool,theoryToolLabel,theoryToolDescription
0,https://knowledge.c-innovationhub.com/heat/702d74d5-df60-4172-91eb-db6f0ff27885,TAM 2 - Technology Acceptance Model 2,"TAM2 is an extension of the original Technology Acceptance Model (TAM), developed by Venkatesh and Davis (2000). It aims to explain user acceptance of technology by incorporating social influence and cognitive instrumental processes."
1,https://knowledge.c-innovationhub.com/heat/7c7845e5-fb69-43e6-91b9-350c68e53303,TAM3 - Technology Acceptance Model 3,The Technology Acceptance Model 3 (TAM3) integrates determinants from the original TAM and TAM2 with factors from the Model of the Determinants of Perceived Ease of Use to explain and predict users’ acceptance and use of technology through both perceived usefulness and perceived ease of use.\n
2,https://knowledge.c-innovationhub.com/heat/a5ce0a8f-6990-43ac-93b7-f8ba3d9fbd8b,AI-TAM - Artificial Intelligence - Technology Acceptance Model,"The AI-TAM model (Artificial Intelligence - Technology Acceptance Model) is an extension of the traditional Technology Acceptance Model (TAM), specifically designed to evaluate user acceptance and collaborative intention in human-in-the-loop AI applications."
3,https://knowledge.c-innovationhub.com/heat/2fe3058c-c9bb-4b62-878d-f735bc7f6661,TAM - Technology Acceptance Model,"The Technology Acceptance Model (TAM) is a widely used theoretical framework that explains how users come to accept and use a technology. Developed by Fred Davis in 1986, TAM posits that two primary constructs—Perceived Usefulness (PU) and Perceived Ease of Use (PEOU)—are key determinants of technology adoption. Perceived Usefulness refers to the degree to which a person believes that using a particular system would enhance their job performance, while Perceived Ease of Use reflects the extent to which a person believes that using the system would be free of effort. These constructs influence users' attitude toward using the technology, which in turn affects their behavioral intention to use it, ultimately leading to actual system use."


Wow, there are different tools that follow the TAM model in the HEAT-KG! Say that the digital system you want to evaluate is based on Artificial Intelligence: it seems that the AI-TAM tool could be interesting for your case. Let's query the HEAT-KG to get more information about AI-TAM, which has the following URI: <https://knowledge.c-innovationhub.com/heat/a5ce0a8f-6990-43ac-93b7-f8ba3d9fbd8b>.

In [84]:
# Execute the query to get the details of the AI-TAM tool
execute_query('tool-details.rq','https://knowledge.c-innovationhub.com/heat/a5ce0a8f-6990-43ac-93b7-f8ba3d9fbd8b')

Substituting URI: https://knowledge.c-innovationhub.com/heat/a5ce0a8f-6990-43ac-93b7-f8ba3d9fbd8b
Executing query: tool-details.rq


,toolLabel,toolDescription,toolPaper,toolContextLabel,toolContext,toolTheoryLabel,toolTheory,toolDiagram,previousToolLabel,previousTool
0,AI-TAM - Artificial Intelligence - Technology Acceptance Model,"The AI-TAM model (Artificial Intelligence - Technology Acceptance Model) is an extension of the traditional Technology Acceptance Model (TAM), specifically designed to evaluate user acceptance and collaborative intention in human-in-the-loop AI applications.",https://doi.org/10.15346/hc.v9i1.134,AI adoption,https://knowledge.c-innovationhub.com/heat/context/a1b4580a-20d9-4419-938f-f0f718a2b7e9,Technology Acceptance Model (TAM),https://knowledge.c-innovationhub.com/heat/ReferenceTheory/df9b0e8a-9819-420e-8f08-d6e478f789c1,https://figures.semanticscholar.org/1055b796992898a75cd27093bc956253da74616e/5-Figure1-1.png,TAM 2 - Technology Acceptance Model 2,https://knowledge.c-innovationhub.com/heat/702d74d5-df60-4172-91eb-db6f0ff27885


Sweet! Now, if you want to learn more about this user research tool, you can go and read the paper at https://doi.org/10.15346/hc.v9i1.134 ;-)

Now, let's proceed to understand which investigation dimensions are included in AI-TAM. To do so, we can execute the query that gets all construct of a tool, passing again the AI-TAM identifier as second parameter.

In [85]:
# Execute the query to get the constructs of the AI-TAM tool
execute_query('constructs-of-tool.rq','https://knowledge.c-innovationhub.com/heat/a5ce0a8f-6990-43ac-93b7-f8ba3d9fbd8b')

Substituting URI: https://knowledge.c-innovationhub.com/heat/a5ce0a8f-6990-43ac-93b7-f8ba3d9fbd8b
Executing query: constructs-of-tool.rq


,construct,constructLabel,constructDescription
0,https://knowledge.c-innovationhub.com/heat/construct/cf224327-48e9-445d-a785-ecf07b03525c,Collaborative Intention (CI),"The user's willingness to actively contribute to the AI process (e.g., validating or correcting outputs)"
1,https://knowledge.c-innovationhub.com/heat/construct/b77d1659-f405-4475-b397-dd95566512b2,Trust in AI (TRUST),The user's confidence in the AI system's reliability and decision-making
2,https://knowledge.c-innovationhub.com/heat/construct/fb0dfcc2-7c4b-4646-911c-336cdcc5de43,Perceived Usefulness (PU),The degree to which a user believes that using the AI system enhances their task performance
3,https://knowledge.c-innovationhub.com/heat/construct/2fb98f94-eb40-4b46-828f-e3abdf27a9f1,Perceived Quality of AI Output (PQ),"The user's evaluation of the accuracy, clarity, and relevance of the AI-generated results"
4,https://knowledge.c-innovationhub.com/heat/construct/dcaf9171-a2ac-49a8-afa5-cf987f52e7db,Behavioral Intention to Use (BI),The likelihood that the user will continue using the AI system in the future
5,https://knowledge.c-innovationhub.com/heat/construct/5a972857-9fbc-4493-b99b-622929c3d9c5,Ease of Use (EOU),How easy and intuitive the user finds the AI system to interact with


Now you know which are the dimensions that you can investigate if you reuse the AI-TAM tool. If, for example, your AI-based system does not include a human-in-the-loop mechanism, you can avoid using the Collaborative Intention construct, but the other constructs may be fine. Now, if you identified the interesting constructs, you can jump to the section on exploring the HEAT-KG starting from a Construct.

### Exploration starting from a General Construct

In case you are not familiar with user research theories, you may still have an idea about what you would like to assess in your questionnaire. In this case, you can start exploring the HEAT-KG starting from the General Constructs. A general construct is the "abstract" definition of an investigation factor or dimension: once you identify the general constructs that are relevant for your user research study, you'll need to look for concrete implementation of construct in existing tools.

Say, for example, that you interested in investigating the attitude of your users towards your digital system. You explore all general constructs in the HEAT-KG to see if there is any such dimension.

In [ ]:
# Execute the query to get all general constructs
execute_query('all-general-constructs.rq')

There are indeed different general constructs containing "attitude" in their name, let's start exploring the one that has exactly that name, which has the following URI: <https://knowledge.c-innovationhub.com/heat/GeneralConstruct/ccaee734-9799-4116-a1e4-4be3ad58111e>.

In [87]:
# execute the query that asks for all details about the Attitude general construct
execute_query('general-construct-details.rq','https://knowledge.c-innovationhub.com/heat/GeneralConstruct/ccaee734-9799-4116-a1e4-4be3ad58111e')

Substituting URI: https://knowledge.c-innovationhub.com/heat/GeneralConstruct/ccaee734-9799-4116-a1e4-4be3ad58111e
Executing query: general-construct-details.rq


,gconstructLabel,gconstructDescription,narrowerGConstruct,narrowerGCLabel,narrowerGCDescription
0,Attitude,"Overall evaluative disposition toward technology or system use, encompassing perceived favorability, beliefs about benefits and risks, emotional responses, and behavioral attitudes",https://knowledge.c-innovationhub.com/heat/GeneralConstruct/ad167627-f826-404c-bcb6-4f7b689f0fb7,Behavioral Attitude,"Likelihood or inclination to perform a behavior, reflecting a person’s predisposition or tendency to act in a certain way, based on prior evaluations and experiences"
1,Attitude,"Overall evaluative disposition toward technology or system use, encompassing perceived favorability, beliefs about benefits and risks, emotional responses, and behavioral attitudes",https://knowledge.c-innovationhub.com/heat/GeneralConstruct/1111b65f-458e-4be8-ab4c-5e216efb5b41,Cognitive Attitude,"Mental component of an attitude, involving a person’s beliefs, thoughts, and knowledge about something (e.g. a system or technology) and reflecting what that person thinks or believes (rather than what they feel or intend to do)"
2,Attitude,"Overall evaluative disposition toward technology or system use, encompassing perceived favorability, beliefs about benefits and risks, emotional responses, and behavioral attitudes",https://knowledge.c-innovationhub.com/heat/GeneralConstruct/6dd579e0-f25f-40ac-b931-9c2f0a32385e,Affective Attitude,"Emotional reactions toward a system or technology, such as enthusiasm, curiosity, fear, or distrust, highlighting positive feelings (e.g., hope, confidence) or negative ones (e.g., anxiety, apprehension)"


It seems indeed that there is a generic concept of "attitude", which can be further specified into "affective", "cognitive" and "behavioural". Depending on the context or on the system you want to evaluate, you may want to investigate one or more of such specific factors. 

Let's say that those narrower meanings are too specific for your goal and you are simply interested in the broader one. You can proceed to find any concrete construct implementing the "attitude" dimension, by running the related query.

In [88]:
# Execute the query to find all constructs implementing the Attitude general construct
execute_query('constructs-of-general-construct.rq','https://knowledge.c-innovationhub.com/heat/GeneralConstruct/ccaee734-9799-4116-a1e4-4be3ad58111e')

Substituting URI: https://knowledge.c-innovationhub.com/heat/GeneralConstruct/ccaee734-9799-4116-a1e4-4be3ad58111e
Executing query: constructs-of-general-construct.rq


,construct,constructLabel,constructDescription,toolLabel,tool
0,https://knowledge.c-innovationhub.com/heat/construct/4f294446-871b-451d-93a3-3ed7040ae6d6,Attitude toward using technology,"Attitude towards using technology reflects whether the user perceives using the system as pleasant, enjoyable, wise, or beneficial, versus unpleasant, foolish, or undesirable.",UTAUT - Unified theory of acceptance and use of technology,https://knowledge.c-innovationhub.com/heat/1ed36590-cfbd-4f3a-9a86-d100b584f531
1,https://knowledge.c-innovationhub.com/heat/construct/fa0e6d8f-226a-416b-9265-e5f5bc4783a7,Attitude,"Attitude is the overall positive or negative evaluation of using the system. It reflects how favourable, desirable, pleasant, or beneficial the behaviour is perceived to be by the individual. Each of the items can be reversed (e.g (1) bad/good, (2) foolish/wise, (3) like/dislike, (4) pleasant/unpleasant",DTPB - Decomposed Theory of Planned Behavior,https://knowledge.c-innovationhub.com/heat/d3fb425a-981d-4119-80d0-cfb8a1391928


Interesting! There are two constructs that implement the idea of attitude and they are from two different tools, namely UTAUT and DTPB. Now, if you are interested in learning more about the tools, you can proceed to ask for more details about each of them, as we did above, and potentially discover additional interesting constructs.

Instead, if you are happy with the selected constructs for your questionnaire, proceed to the next type of exploration.

### Exploration starting from a Construct

Let's say that, through the types of explorations above, you have discovered what constructs you were looking for and now you would like to proceed to put together your user research study. You need to check the items for your questionnaire.

Imagine that you have selected the following constructs:
- System Usability (https://knowledge.c-innovationhub.com/heat/construct/6733a1f8-3bf6-43cb-817c-799f4e9e7b73) from the SUS tool
- Mental Workload (https://knowledge.c-innovationhub.com/heat/construct/762fb388-9220-4309-90e0-1b4454c24c17) from the NASA-TLX tool
- Perceived Usefulness (https://knowledge.c-innovationhub.com/heat/construct/950a10c3-071b-4183-9c72-adbffb5afe2a) and Behavioural Intention to Use (https://knowledge.c-innovationhub.com/heat/construct/7bcd7112-4dd3-4690-9fa8-e11c5a4f885e) from the TAM 2 tool

For each construct, you can now explore the HEAT-KG and check the items, their formulation, the answer options and their scale, to include them in the questionnaire. Let's run the query that gives all questions of each construct.

In [89]:
# Dictionary of the selected constructs (with names and URIs)
constructs = {
    'System Usability': 'https://knowledge.c-innovationhub.com/heat/construct/6733a1f8-3bf6-43cb-817c-799f4e9e7b73',
    'Mental Workload': 'https://knowledge.c-innovationhub.com/heat/construct/762fb388-9220-4309-90e0-1b4454c24c17',
    'Perceived Usefulness': 'https://knowledge.c-innovationhub.com/heat/construct/950a10c3-071b-4183-9c72-adbffb5afe2a',
    'Behavioural Intention to Use': 'https://knowledge.c-innovationhub.com/heat/construct/7bcd7112-4dd3-4690-9fa8-e11c5a4f885e',
}

# Loop through each construct and execute the query to get the questions and their Likert scales
for construct_name, construct_uri in constructs.items():
    print(f"\n{'='*70}")
    print(f"Construct: {construct_name}")
    print(f"{'='*70}\n")
    
    results = execute_query('questions-of-construct.rq', construct_uri)
    display(results)


Construct: System Usability

Substituting URI: https://knowledge.c-innovationhub.com/heat/construct/6733a1f8-3bf6-43cb-817c-799f4e9e7b73
Executing query: questions-of-construct.rq


,questionText,questionOptionsText,questionOptionsValues,questionReversed
0,I would imagine that most people would learn to use this system very quickly,[Strongly Disagree-Strongly Agree],[1-5],false
1,I thought the system was easy to use,[Strongly Disagree-Strongly Agree],[1-5],false
2,I think that I would need the support of a technical person to be able to use this system,[Strongly Disagree-Strongly Agree],[1-5],true
3,I needed to learn a lot of things before I could get going with this system,[Strongly Disagree-Strongly Agree],[1-5],true
4,I felt very confident using the system,[Strongly Disagree-Strongly Agree],[1-5],false
5,I found the system unnecessarily complex,[Strongly Disagree-Strongly Agree],[1-5],true
6,I found the system very cumbersome to use,[Strongly Disagree-Strongly Agree],[1-5],true
7,I found the various functions in this system were well integrated,[Strongly Disagree-Strongly Agree],[1-5],false
8,I think that I would like to use this system frequently,[Strongly Disagree-Strongly Agree],[1-5],false
9,I thought there was too much inconsistency in this system,[Strongly Disagree-Strongly Agree],[1-5],true



Construct: Mental Workload

Substituting URI: https://knowledge.c-innovationhub.com/heat/construct/762fb388-9220-4309-90e0-1b4454c24c17
Executing query: questions-of-construct.rq


,questionText,questionOptionsText,questionOptionsValues,questionReversed
0,"(Mental Demand)How much mental and perceptual activity was required (e.g., thinking, deciding, calculating, remembering, looking, searching, etc.)? Was the task easy or demanding, simple or complex, exacting or forgiving?",[Low-High],[1-7],false
1,(Effort) How hard did you have to work (mentally and physically) to accomplish your level of performance?,[Low-High],[1-7],false
2,(Performance) How successful do you think you were in accomplishing the goals of the task set by the experimenter (or yourself)? How satisfied were you with your performance in accomplishing these goals?,[Poor-Good],[1-7],true
3,"(Physical demand): How much physical activity was required (e.g., pushing, pulling, turning, controlling, activating, etc.)? Was the task easy or demanding, slow or brisk, slack or strenuous, restful or laborious?",[Low-High],[1-7],false
4,"(Frustration level) How insecure, discouraged, irritated, stressed, and annoyed versus secure, gratified, content, relaxed, and complacent did you feel during the task?",[Low-High],[1-7],false
5,(Temporal demand) How much time pressure did you feel due to the rate or pace at which the tasks or task elements occurred? Was the pace slow and leisurely or rapid and frantic?,[Low-High],[1-7],false



Construct: Perceived Usefulness

Substituting URI: https://knowledge.c-innovationhub.com/heat/construct/950a10c3-071b-4183-9c72-adbffb5afe2a
Executing query: questions-of-construct.rq


,questionText,questionOptionsText,questionOptionsValues,questionReversed
0,I find the system useful in my job,[Strongly Disagree-Strongly Agree],[1-7],false
1,Using the system improves my job performance,[Strongly Disagree-Strongly Agree],[1-7],false
2,Using the system increases my productivity,[Strongly Disagree-Strongly Agree],[1-7],false
3,Using the system enhances my effectiveness,[Strongly Disagree-Strongly Agree],[1-7],false



Construct: Behavioural Intention to Use

Substituting URI: https://knowledge.c-innovationhub.com/heat/construct/7bcd7112-4dd3-4690-9fa8-e11c5a4f885e
Executing query: questions-of-construct.rq


,questionText,questionOptionsText,questionOptionsValues,questionReversed
0,I plan to use the system frequently,[Strongly Disagree-Strongly Agree],[1-7],false
1,I intend to use the system in the future,[Strongly Disagree-Strongly Agree],[1-7],false
2,I predict I will use the system regularly,[Strongly Disagree-Strongly Agree],[1-7],false


It seems there are indeed several interesting questionnaire items that you can reuse in your user research! 

Now, if you want to base your work on literature, you should reuse the items exactly as they are in the original works; this would allow you to compare your results with other studies and you can be confident that the reliability and validity of your assessment tool is assured. 

Nonetheless, you may find that some of the questions are less relevant for your context: you can think about selecting only a subset of the items for each of the construct (usually you should keep at least 3 items per construct). 

Please note that, instead, rephrasing or reformulating the items may change the meaning or the interpretation of your respondents, so be as close as possible to the original expression; if you need your questionnaire in a language different from English, you can search online, because for the most commonly used tools there indeed exist official translations in multiple languages.

Good luck with your user research!